In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
import sys

sys.path.append('../scripts')
sys.path.append('../')

from scripts import graph_osm_loader
from scripts import pipeline, centroids_graph_builder, clustering
from scripts.path_findings import excraction_pfa, dijkstra_pfa
from scripts import utils
import scripts

In [3]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

723 1046


In [4]:
points = utils.read_points(GRAPH_ID, g, num=1000)

In [5]:
nodes = g.nodes()

In [6]:
import numpy as np


def dst(da, db):
    return ((da['x'] - db['x']) ** 2 + (da['y'] - db['y']) ** 2) ** 0.5 / 360 * 2 * np.pi * 6371.01 * 1000 * 0.56


def node_dst(u, v):
    da = nodes[u]
    db = nodes[v]
    return dst(da, db)

In [10]:
from scripts.path_findings.alt_builder import RandomAltBuilder, ClusterAltBuilder

cms_resolver = clustering.LouvainCommunityResolver(resolution=25)
cg = centroids_graph_builder.CentroidGraphBuilder().build(g, cms_resolver)

baseline = dijkstra_pfa.Dijkstra(g)

# test_alg = excraction_pfa.ExtractionPfa(
#     g = g,
#     cg=cg,
#     upper=dijkstra_pfa.Dijkstra(cg.g),
#     down=dijkstra_pfa.Dijkstra(g)
# )


# test_alg = dijkstra_pfa.AStar(
#     g = g,
#     h = node_dst
# )
test_alg = ClusterAltBuilder(1).build_ch_pfa(g)

find centroids:   0%|          | 0/143 [00:00<?, ?it/s]

find edges:   0%|          | 0/143 [00:00<?, ?it/s]

2024-12-23 15:22:58,638 - root - INFO - cms: 26


  0%|          | 0/723 [00:00<?, ?it/s]

In [11]:
stat = pipeline.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=4
).compare()

start 0


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 2


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 3


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

In [12]:
stat


        err_max:    0.0000
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   0.0000
        
        acceleration_max:    6.5789
        acceleration_min:    0.1615
        acceleration_median: 1.1094
        acceleration_mean:   1.2626
        